### Day 15: Warehouse Woes

Link: https://adventofcode.com/2024/day/15#part2

To solve part two, we need to adapt the solution from part one to check whether the other half of the box can be moved and, if so, to move it afterwards. Given that boxes are 1 unit high and 2 units wide, these extra steps are only necessary when the robot is moving vertically, pushing the boxes up or down. In these situations, we also need to track the cells we have already processed, as it is possible to add the same half of the box for processing more than once.

To calculate the GPS score, we can count the occurrences of the left side of the box, "[", at the end of the simulation.

In [ ]:
# Please ensure there is an `input.txt` file in this folder containing your input.
with open("input.txt", "r") as file:
    lines = file.readlines()

In [ ]:
import typing as t
from dataclasses import dataclass
from collections import deque


@dataclass
class Position:
    row: int
    column: int


@dataclass
class Robot:
    position: Position


grid: list[list[int]] = []


for row, line in enumerate(lines):
    line = line.strip()

    if not line:
        break

    grid_row: list[str] = []

    for column, cell in enumerate(line):
        if cell == ".":
            grid_row.extend([".", "."])
        elif cell == "#":
            grid_row.extend(["#", "#"])
        elif cell == "O":
            grid_row.extend(["[", "]"])
        elif cell == "@":
            grid_row.extend(["@", "."])

    grid.append(grid_row)


def find_robot(grid: list[list[str]]) -> Robot:
    for row in range(len(grid)):
        for column, cell in enumerate(grid[row]):
            if cell == "@":
                return Robot(position=Position(row=row, column=column))

    raise Exception("Robot not found")


robot = find_robot(grid)
commands: list[str] = []


for row in range(row + 1, len(lines)):
    line = lines[row].strip()
    commands.extend(list(line))


actions = {
    "^": (-1, 0),
    "v": (1, 0),
    "<": (0, -1),
    ">": (0, 1),
}


def make_command_new_row_column(command: str, row: int, column: int) -> tuple[int, int]:
    return row + actions[command][0], column + actions[command][1]


def can_update_grid(
    command: str, row: int, column: int, cache: t.Optional[dict[tuple[int, int], bool]]
) -> bool:
    if cache is None:
        cache = {}

    if (row, column) in cache:
        return cache[row, column]

    if grid[row][column] == "#":
        return False

    if grid[row][column] == ".":
        return True

    new_row, new_column = make_command_new_row_column(command, row, column)
    can_update = can_update_grid(command, new_row, new_column, cache)
    cache[(new_row, new_column)] = can_update

    if can_update and command in {"^", "v"} and grid[new_row][new_column] in {"[", "]"}:
        add_column = 1 if grid[new_row][new_column] == "[" else -1
        new_column += add_column
        can_update = can_update_grid(command, new_row, new_column, cache)
        cache[(new_row, new_column)] = can_update

    return can_update


def update_grid(command: str, robot: Robot) -> None:
    to_update = deque([(".", robot.position.row, robot.position.column)])
    updated: set[tuple[int, int]] = set()

    while to_update:
        new_cell_value, row, column = to_update.popleft()

        if (row, column) in updated:
            continue

        updated.add((row, column))
        old_cell_value = grid[row][column]
        grid[row][column] = new_cell_value

        if new_cell_value == "@":
            robot.position.row = row
            robot.position.column = column

        if old_cell_value in {".", "#"}:
            continue

        new_row, new_column = make_command_new_row_column(command, row, column) 

        if command in {"^", "v"} and old_cell_value in {"[", "]"}:
            add_column = 1 if old_cell_value == "[" else -1
            to_update.append((".", row, column + add_column))

        to_update.append((old_cell_value, new_row, new_column))


def print_grid() -> None:
    import time

    with open("output.txt", "w") as file:
        grid_lines = "\n".join(["".join(grid_row) for grid_row in grid])
        file.writelines(grid_lines)

    time.sleep(.5)


for command in commands:
    if can_update_grid(command, robot.position.row, robot.position.column, None):
        update_grid(command, robot)
        # print_grid()


sum_coordinates = 0


for row in range(len(grid)):
    for column, cell in enumerate(grid[row]):
        if cell == "[":
            sum_coordinates += row * 100 + column


print(sum_coordinates)